In [14]:
import boto3
s3_client = boto3.client('s3')

s3_client.download_file(Bucket='pinterestkafkabucket',Key='2022-02-24 21:02:20.json',Filename='./2022-02-24 21:02:20.json')

In [7]:
!wget 's3://pinterestkafkabucket/2022-02-23 20:42:07.json'

s3://pinterestkafkabucket/2022-02-23 20:42:07.json: Unsupported scheme ‘s3’.


In [15]:
import json

with open('2022-02-24 21:02:20.json', mode='r') as json_file:
    #print(type(json_file))
    #data = json.loads(json_file.read())
    data = json.load(json_file)
    #print(type(data))
    #ast.literal_eval(data)

print(data)
print(type(data))




{'category': 'travel', 'index': 10020, 'unique_id': '56cac475-b169-42cd-94b0-16978dff21d8', 'title': '10 Tips for Planning a Trip to Scotland and Ireland', 'description': 'Ten tips to keep in mind when planning a trip to Scotland and Ireland to make sure you have the best travel experience possible!', 'follower_count': '5k', 'tag_list': 'Scotland Vacation,Scotland Road Trip,Ireland Vacation,Scotland Travel,Scotland Uk,Italy Vacation,Emerald Isle,Places To Travel,Travel Destinations', 'is_image_or_video': 'image', 'image_src': 'https://i.pinimg.com/originals/71/ac/1e/71ac1e87ac36489ce1fe4695c279ec26.png', 'downloaded': 1, 'save_location': 'Local save in /data/travel'}
<class 'dict'>


In [ ]:
# Three ways to clean the data
# 1. Tag list - put into list comma delimit
# 2. is image_or_video - boolean
# 3. re-order? UUID first?

# Spark connecting to S3

In [1]:
import json
import boto3
import pyspark
import findspark
import multiprocessing
findspark.init('/home/martin96/Spark/spark-3.2.1-bin-hadoop3.2')
s3_client = boto3.client('s3')
sessions3 = boto3.Session()
s3 = sessions3.resource('s3')
my_bucket = s3.Bucket('pinterestkafkabucket')

conf = (
    pyspark.SparkConf()
    # Setting where master node is located [cores for multiprocessing]
    .setMaster(f"local[{multiprocessing.cpu_count()}]")
    # Setting application name
    .setAppName("TestApp")
    # Setting config value via string
    .set("spark.eventLog.enabled", False)
    # Setting environment variables for executors to use
    .setExecutorEnv(pairs=[("VAR3", "value3"), ("VAR4", "value4")])
    # Setting memory if this setting was not set previously
    .setIfMissing("spark.executor.memory", "1g")
)
sessionsp = pyspark.sql.SparkSession.builder.config(conf=conf).getOrCreate()
sc = sessionsp.sparkContext


22/03/02 20:28:25 WARN Utils: Your hostname, martin96-TP300LA resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface wlp2s0)
22/03/02 20:28:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 20:28:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [76]:
import pandas as pd
from functools import reduce
#df = sessionsp.createDataFrame(pkeys.collect())
df_list = []
for obj in my_bucket.objects.all():
### Read streaming body
    obj = s3_client.get_object(Bucket='pinterestkafkabucket', Key=obj.key)
    content = obj['Body'].read()
    data = json.loads(content)
    pkeys = sc.parallelize([data])
    norm = pd.json_normalize(pkeys.collect())
    norm_list = norm.iloc[0].tolist()
    df = sessionsp.createDataFrame(pd.DataFrame(norm, columns=norm.keys()))
    df_list.append(df)
df = reduce(pyspark.sql.DataFrame.union, df_list)
#df.show()
df.select("category").show()

+--------------+
|      category|
+--------------+
|        travel|
|           art|
|diy-and-crafts|
|      vehicles|
|     education|
|  mens-fashion|
|        travel|
|     education|
|           art|
|     christmas|
|diy-and-crafts|
+--------------+



In [67]:
#n = pd.DataFrame(norm)
# obj = json.loads(norm)
# pairs = obj.items()
# it_list = []
# for k, v in pairs:
#     it_list.append(v)
# print(it_list)
#norm.iloc[0].tolist()
pd.DataFrame(norm, columns=norm.keys())


,category,index,unique_id,title,description,follower_count,tag_list,is_image_or_video,image_src,downloaded,save_location
0,diy-and-crafts,3032,208b07bc-e042-4e84-8855-97905f691848,Puffy Ghost Craft,This ghost craft is a super simple Halloween c...,56k,"Halloween Arts And Crafts,Halloween Crafts For...",video,https://i.pinimg.com/videos/thumbnails/origina...,1,Local save in /data/diy-and-crafts


In [1]:
import json
import boto3
import pyspark
import findspark
findspark.init('/home/martin96/Spark/spark-3.2.1-bin-hadoop3.2')
s3_client = boto3.client('s3')

### Read streaming body
obj = s3_client.get_object(Bucket='pinterestkafkabucket', Key="2022-02-24 21:02:20.json")
content = obj['Body'].read()
data = json.loads(content)
# my_list = []
# for v, k in zip(data.values(), data.keys()):
#     my_list.append(k)
#     my_list.append(v)

#print(my_list)
###
# Get a Spark context and use it to parallelize the keys
conf = pyspark.SparkConf().setAppName("PinterestApp")
sc = pyspark.SparkContext(conf=conf)
pkeys = sc.parallelize([data]) #keys)
print(pkeys)

22/03/02 19:59:34 WARN Utils: Your hostname, martin96-TP300LA resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface wlp2s0)
22/03/02 19:59:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 19:59:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274


In [11]:
#rdd2 = sc.parallelize(list(pkeys.items()))
#rdd3 = pkeys.map(lambda x: dict([x]))
pkeys.count()

1

In [47]:
print(data)

{'category': 'travel', 'index': 10020, 'unique_id': '56cac475-b169-42cd-94b0-16978dff21d8', 'title': '10 Tips for Planning a Trip to Scotland and Ireland', 'description': 'Ten tips to keep in mind when planning a trip to Scotland and Ireland to make sure you have the best travel experience possible!', 'follower_count': '5k', 'tag_list': 'Scotland Vacation,Scotland Road Trip,Ireland Vacation,Scotland Travel,Scotland Uk,Italy Vacation,Emerald Isle,Places To Travel,Travel Destinations', 'is_image_or_video': 'image', 'image_src': 'https://i.pinimg.com/originals/71/ac/1e/71ac1e87ac36489ce1fe4695c279ec26.png', 'downloaded': 1, 'save_location': 'Local save in /data/travel'}


In [9]:
my_list = []
for v, k in zip(data.values(), data.keys()):
    my_list.append(k)
    my_list.append(v)


my_dict = dict(zip(my_list[::2], my_list[1::2]))
print(my_dict)


{'category': 'travel', 'index': 10020, 'unique_id': '56cac475-b169-42cd-94b0-16978dff21d8', 'title': '10 Tips for Planning a Trip to Scotland and Ireland', 'description': 'Ten tips to keep in mind when planning a trip to Scotland and Ireland to make sure you have the best travel experience possible!', 'follower_count': '5k', 'tag_list': 'Scotland Vacation,Scotland Road Trip,Ireland Vacation,Scotland Travel,Scotland Uk,Italy Vacation,Emerald Isle,Places To Travel,Travel Destinations', 'is_image_or_video': 'image', 'image_src': 'https://i.pinimg.com/originals/71/ac/1e/71ac1e87ac36489ce1fe4695c279ec26.png', 'downloaded': 1, 'save_location': 'Local save in /data/travel'}


In [14]:
def pairwise(iterable):
    a = iter(iterable)
    return zip(a, a)

#pkeys = sc.parallelize(data)
act = dict(pairwise(pkeys.collect()))
#pkeys.saveAsTextFile('./')

{'category': 'travel', 'index': 10020, 'unique_id': '56cac475-b169-42cd-94b0-16978dff21d8', 'title': '10 Tips for Planning a Trip to Scotland and Ireland', 'description': 'Ten tips to keep in mind when planning a trip to Scotland and Ireland to make sure you have the best travel experience possible!', 'follower_count': '5k', 'tag_list': 'Scotland Vacation,Scotland Road Trip,Ireland Vacation,Scotland Travel,Scotland Uk,Italy Vacation,Emerald Isle,Places To Travel,Travel Destinations', 'is_image_or_video': 'image', 'image_src': 'https://i.pinimg.com/originals/71/ac/1e/71ac1e87ac36489ce1fe4695c279ec26.png', 'downloaded': 1, 'save_location': 'Local save in /data/travel'}


In [10]:
# import json
# #print(obj['Body'])
# with open(obj['Body'], mode='r') as json_file:
#     data = json.load(json_file)
# print(data)
pkeys = sc.parallelize(obj['Body'])
pkeys = pkeys.countByValue()
print(pkeys)

FileNotFoundError: [Errno 2] No such file or directory: "defaultdict(<class 'int'>, {})"

In [4]:
session = boto3.Session()

In [5]:
s3 = session.resource('s3')
my_bucket = s3.Bucket('pinterestkafkabucket')

for obj in my_bucket.objects.all():
    print(obj.key)

2022-02-24 21:02:20.json
2022-02-24 21:02:23.json
2022-02-24 21:02:24.json
2022-02-24 21:02:25.json
2022-02-24 21:02:26.json
2022-02-24 21:02:28.json
2022-02-24 21:02:30.json
2022-02-24 21:02:31.json
2022-02-24 21:02:32.json
2022-02-24 21:02:34.json
2022-02-24 21:02:36.json


In [18]:
s3_client = boto3.client('s3')
my_objs = s3_client.list_objects(Bucket='pinterestkafkabucket')

print(my_objs.keys())
my_objs['Contents'][0]['Key']

dict_keys(['ResponseMetadata', 'IsTruncated', 'Marker', 'Contents', 'Name', 'Prefix', 'MaxKeys', 'EncodingType'])


'2022-02-24 21:02:20.json'